## Synthesizing output from matches

In [4]:
from sppysound.database import AudioDatabase, Synthesizer, Matcher
import synthesis_config
import config

In [5]:
source_dir = "./ExampleDatabase"
target_dir = "./ExampleTarget"
output_dir = "./ExampleOutput"

Load source database.
Also load the F0, RMS and Peak analyses for use with amplitude and pitch enforcement.

In [6]:
source_database = AudioDatabase(
    source_dir,
    config=synthesis_config,
    analysis_list={"f0", "rms"}
)
source_database.load_database(reanalyse=True)

Load database used to generate matches to source database. 
This is used when enforcing analyses such as RMS and F0. (Original grains are needed to calculate the ratio to alter the synthesized grain by)

In [7]:
target_database = AudioDatabase(
    target_dir,
    config=synthesis_config,
    analysis_list={"f0", "rms"}
)
target_database.load_database(reanalyse=True)

output_database = AudioDatabase(
    output_dir,
    config=config
)
output_database.load_database(reanalyse=False)

matcher = Matcher(
    source_database,
    target_database,
    output_db=output_database,
    config=config,
    rematch=True
)
matcher.match(
    matcher.kdtree_matcher,
    grain_size=config.matcher["grain_size"],
    overlap=config.matcher["overlap"]
)

Traceback (most recent call last):
  File "/Users/samuelperry/PerryPerrySource/pysource/sppysound/src/sppysound/database.py", line 157, in analyse_database
    config=self.config
  File "/Users/samuelperry/PerryPerrySource/pysource/sppysound/src/sppysound/audiofile.py", line 943, in __enter__
    "empty".format(self.name))
IOError: File isn't valid: ElectricGuitarSample-out_output.wav
Check that file is mono and isn't empty


Initialise the synthesizer object used for generating the final output.

In [8]:
synthesizer = Synthesizer(
    source_database, 
    output_database, 
    target_db=target_database, 
    config=config
)

Run synthesis. As with the matching, warnings may be generated. These have all been accounted for and will be silenced in a future release. The output audio can now be found in the audio folder of ./ExampleOutput

In [9]:
synthesizer.synthesize(
    grain_size=config.synthesizer["grain_size"],
    overlap=config.synthesizer["overlap"]
)

The synthesis_config.py file for this demo is:

~~~python
rms = {
    "window_size": 100,
    "overlap": 2,
}

analysis_dict = {
    "f0": "log2_median",
    "rms": "mean"
}

analysis = {
    "reanalyse": False
}

output_file = {
    "samplerate": 44100,
    "format": 131075,
    "channels": 1
}

synthesizer = {
    "enforce_rms": True,
    "enf_rms_ratio_limit": 5.,
    "enforce_f0": True,
    "enf_f0_ratio_limit": 10.,
    "grain_size": 100,
    "overlap": 2,
    "normalize" : True,
    # Defines the number of potential grains to choose from matches when
    # synthesizing output.
    "match_quantity": 20
}
~~~